In [29]:
import pandas as pd
import numpy as np

## Task 2

In [30]:
result = pd.read_csv('assignments_from_pool_29960711__05-12-2021.tsv', sep='\t')

In [31]:
def link2id(x):
    return x.split('/')[-1].split('"')[0]

result['INPUT:image'] = result['INPUT:image'].apply(link2id)

In [32]:
result['GOLDEN:result'] = result['GOLDEN:result'].fillna(1.)

In [33]:
generated_img_result = result[result['GOLDEN:result'] == 1.0]

In [35]:
aggregated_result = generated_img_result.groupby('INPUT:image').sum()

In [37]:
final_result = aggregated_result[['OUTPUT:gender', 'OUTPUT:result', 'OUTPUT:confidence', 'GOLDEN:result']]

In [41]:
final_result = final_result.rename({
    'OUTPUT:gender': 'gender', 
    'OUTPUT:result': 'result',
    'OUTPUT:confidence': 'confidence', 
    'GOLDEN:result': 'n_workers'
}, axis=1)

In [43]:
final_result['n_workers'] = final_result['n_workers'].astype('int')

workers do not always agree on genders of the image: indicating random selections

In [62]:
def control_consensus(df):
    # male or female
    return sum((df['gender'] == df['n_workers']) | (df['gender'] == 0)) / len(df)

In [63]:
control_consensus(final_result)

0.9390756302521008

images considered to be `real` by workers with more than 2 votings

In [64]:
cond_1 = (final_result['result'] == 0) & (final_result['n_workers'] >= 2)
cond_2 = (final_result['result'] == 1) & (final_result['n_workers'] == 3)

real_img = pd.concat((final_result[cond_1], final_result[cond_2]))

In [65]:
len(final_result[cond_1])

54

In [66]:
len(final_result[cond_2])

32

workers do not always agree with the gender

In [67]:
control_consensus(real_img)

0.9534883720930233

rate for successfully deceive more than 2 workers

In [69]:
n_real_imgs = len(real_img)
n_valid_imgs = len(final_result[final_result['n_workers'] >= 2])
f"decieved imgs: {n_real_imgs}; imgs with more than 2 workers: {n_valid_imgs} rate: {n_real_imgs / n_valid_imgs}"

'decieved imgs: 86; imgs with more than 2 workers: 407 rate: 0.2113022113022113'